# <span style="background-color:white;color:red;"> ⌛⌛ EN PROGRESO. NO ES ÚTIL⌛⌛</span>

## Bloque I
Descomprimir el csv en formato gz

In [ ]:
"""  
Importación de ficheros de AIRBNB. Preprocesado y cargado a la base de datos 

Obtenemos los datos desde https://insideairbnb.com/get-the-data/

Podríamos hacer scraping, obtener la fecha de la última actualización de Madrid y generar el enlace para descargarlo, pero no es necesario por que se actualiza cada mucho tiempo y no es necesario tener la última versión. Podemos cambiar la info procesable en la carpeta data manualmente.

"""
import os
import pandas as pd
import numpy as np
import datetime
from config_bd import *
import requests
import sys
import gzip
import shutil

DATA_DIR = os.path.join('data')
# DATA_DIR = os.path.join(os.path.dirname(__file__), 'data')
TEMP_DIR = os.path.join(DATA_DIR, 'TEMP')


""" Comprobamos que existe el directorio de datos """

if not os.path.exists(DATA_DIR):
    print(f"El directorio {DATA_DIR} no existe. No es posible continuar...")
    sys.exit()

""" Descomprimimos el archivo de calendario """

if not os.path.exists(TEMP_DIR):
    os.makedirs(TEMP_DIR)


gz_file_path = os.path.join(DATA_DIR, 'calendar.csv.gz')
output_file_path = os.path.join(TEMP_DIR, os.path.splitext(os.path.basename(gz_file_path))[0])

if os.path.exists(gz_file_path):
    try:
        with gzip.open(gz_file_path, 'rb') as f_in:
            with open(output_file_path, 'wb') as f_out:
                shutil.copyfileobj(f_in, f_out)
        print(f"Archivo {gz_file_path} descomprimido como {output_file_path}")
    except Exception as e:
        print(f"Error al descomprimir el archivo {gz_file_path}: {e}")
        sys.exit()
else:
    print(f"El archivo {gz_file_path} no existe. No es posible continuar...")
    sys.exit()


## Bloque II
Carga del csv en pandas. Cambios de tipos e imputación

In [ ]:
df = pd.read_csv(output_file_path, low_memory=False)

In [ ]:
df['price'] = df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)
df['price'] = df['price'].fillna(0)
df['available'] = df['available'].replace({'t': 1, 'f': 0})
df['minimum_nights'] = df['minimum_nights'].fillna(df['minimum_nights'].median()).astype(int)
df['maximum_nights'] = df['maximum_nights'].fillna(df['maximum_nights'].median()).astype(int)
df = df.drop(columns=['adjusted_price'])
df = df.rename(columns={'listing_id': 'property_id'})
# df['fecha'] = pd.to_datetime(df['fecha'], format='%Y-%m-%d')

In [ ]:
df.describe().T

In [ ]:
df.sample(10)

In [ ]:
df.describe()

## Bloque III
Importamos a la base de datos. 
**Requiere haber creado la tabla** <code>01_CREATE TABLE bnb_calendar.sql</code>

In [ ]:
from config_bd import get_connection

In [ ]:
df.to_csv(os.path.join(TEMP_DIR,'transform_calendar.csv'), index=False)

In [ ]:
from tqdm import tqdm
from time import sleep

CHUNKSIZE = 10000
for chunk in tqdm(
    pd.read_csv(os.path.join(TEMP_DIR, 'transform_calendar.csv'), chunksize=CHUNKSIZE, dtype=str),
    unit_scale=CHUNKSIZE, unit=" filas"
):
    sleep(5)




Elimino el fichero para no dejar duplicado el csv

In [ ]:
os.remove(os.path.join(TEMP_DIR,'transform_calendar.csv'))